# Zeek ML task

## Environment setup

This project uses `uv` to manage the Python environment. To set it up, run the following commands in the terminal:

```bash
uv venv
source .venv/bin/activate
```
Then, you may open this notebook in an editor such as VS Code, or launch Jupyter Lab with `jupyter lab` and open the notebook there.

## Task approach

Note: I do not have extensive experience with raw ML models handling network traffic, and have never worked with Zeek outputs (I have far more experience with Suricata). I will mainly use the techniques I deem to be sufficient from my past experiences and ML courses and hope that would be enough for the task.

We do have a low amount of data available, and know, which data are guaranteed to be "good" (the first 4 minutes), but we do not have any labels for data after that; we could train a supervised model on the first 4 minutes, but together with the fact that we do not have any negative examples and a low amount of training data, it would be prone to overfitting. Therefore I argue that in this case a supervised approach would not be an ideal choice.

Instead, we can approach this problem as "novelty detection", train an unsupervised model on the first 4 minutes of data, and then either calculate an anomaly score, or use a classifier to mark data that are dissimilar to the data we found in the first 4 minutes. I will mainly focus on using `OneClassSVM` (with grid search over kernels) and compare it with `IsolationForest`.


I then looked for a way to load Zeek data into a Pandas dataframe, by googling "zeek log to pandas dataframe", a stratosphereips repo was the first thing that showed up. https://github.com/stratosphereips/zeeklog2pandas.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.model_selection import ParameterGrid, KFold

from zeeklog2pandas import read_zeek

Let's use it to load the data and see what we are working with.

In [2]:
df = read_zeek(path="zeek/conn.log")
df.info()
display(df[0:5])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223 entries, 0 to 222
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ts              223 non-null    float64
 1   uid             223 non-null    object 
 2   id.orig_h       223 non-null    object 
 3   id.orig_p       223 non-null    int64  
 4   id.resp_h       223 non-null    object 
 5   id.resp_p       223 non-null    int64  
 6   proto           223 non-null    object 
 7   service         223 non-null    object 
 8   duration        223 non-null    object 
 9   orig_bytes      223 non-null    object 
 10  resp_bytes      223 non-null    object 
 11  conn_state      223 non-null    object 
 12  local_orig      223 non-null    object 
 13  local_resp      223 non-null    object 
 14  missed_bytes    223 non-null    int64  
 15  history         223 non-null    object 
 16  orig_pkts       223 non-null    int64  
 17  orig_ip_bytes   223 non-null    int

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,...,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents
0,1.599057e+09,CMLZbu3FDJYoZwa27k,10.8.0.117,1210,8.8.8.8,53,udp,dns,0.010542,31,...,SF,-,-,0,Dd,1,59,1,108,-
1,1.599057e+09,CgRCjV3z8dKmNVIvhb,10.8.0.117,43814,8.8.8.8,53,udp,dns,0.010908,33,...,SF,-,-,0,Dd,1,61,1,115,-
2,1.599057e+09,CgFfWv3PUApAZUINNf,10.8.0.117,51631,8.8.8.8,53,udp,dns,0.010734,50,...,SF,-,-,0,Dd,1,78,1,94,-
3,1.599057e+09,CIkFu02IznJPZcp1El,10.8.0.117,65449,8.8.8.8,53,udp,dns,0.010405,37,...,SF,-,-,0,Dd,1,65,1,119,-
4,1.599057e+09,Cylq6E2mc9lVLjs8ua,10.8.0.117,63247,8.8.8.8,53,udp,dns,0.009332,40,...,SF,-,-,0,Dd,1,68,1,108,-


Zeek uses `"-"` to represent missing or unset values in its log files. We replace these with `None` so that Pandas can treat them as proper null values.

In [3]:
df.replace("-", None, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223 entries, 0 to 222
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ts              223 non-null    float64
 1   uid             223 non-null    object 
 2   id.orig_h       223 non-null    object 
 3   id.orig_p       223 non-null    int64  
 4   id.resp_h       223 non-null    object 
 5   id.resp_p       223 non-null    int64  
 6   proto           223 non-null    object 
 7   service         156 non-null    object 
 8   duration        190 non-null    object 
 9   orig_bytes      190 non-null    object 
 10  resp_bytes      190 non-null    object 
 11  conn_state      223 non-null    object 
 12  local_orig      0 non-null      object 
 13  local_resp      0 non-null      object 
 14  missed_bytes    223 non-null    int64  
 15  history         214 non-null    object 
 16  orig_pkts       223 non-null    int64  
 17  orig_ip_bytes   223 non-null    int

Some columns are entirely empty across all rows so we can drop them without any issue.

In [4]:
df.drop(columns=["local_orig", "local_resp", "tunnel_parents"], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223 entries, 0 to 222
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ts             223 non-null    float64
 1   uid            223 non-null    object 
 2   id.orig_h      223 non-null    object 
 3   id.orig_p      223 non-null    int64  
 4   id.resp_h      223 non-null    object 
 5   id.resp_p      223 non-null    int64  
 6   proto          223 non-null    object 
 7   service        156 non-null    object 
 8   duration       190 non-null    object 
 9   orig_bytes     190 non-null    object 
 10  resp_bytes     190 non-null    object 
 11  conn_state     223 non-null    object 
 12  missed_bytes   223 non-null    int64  
 13  history        214 non-null    object 
 14  orig_pkts      223 non-null    int64  
 15  orig_ip_bytes  223 non-null    int64  
 16  resp_pkts      223 non-null    int64  
 17  resp_ip_bytes  223 non-null    int64  
dtypes: float64

Now we deal with missing values.

In [5]:
print(df["service"].unique())

['dns' 'ssl' None 'http']


In [6]:
df["service"] = df["service"].fillna("unknown")

From now on, there are multiple approaches we can take. For me it makes sense to treat the 4 minutes that we have as the only data we have available, prepare this data and train a model on them, and then we can run it on the remaining data and manually assess the results, therefore not inadvertedly leaking any information about the remaining data solely by data preprocessing and model design. 

In [7]:

def train_test_split(df: pd.DataFrame):
    ts_min = df["ts"].min()
    train_data = df[df["ts"] < ts_min + 4*60]
    test_data = df[df["ts"] >= ts_min + 4*60]
    return train_data, test_data

train_data, test_data = train_test_split(df)

In [8]:
display(train_data)
display(train_data.describe())
display(df.nunique())

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes
0,1.599057e+09,CMLZbu3FDJYoZwa27k,10.8.0.117,1210,8.8.8.8,53,udp,dns,0.010542,31,80,SF,0,Dd,1,59,1,108
1,1.599057e+09,CgRCjV3z8dKmNVIvhb,10.8.0.117,43814,8.8.8.8,53,udp,dns,0.010908,33,87,SF,0,Dd,1,61,1,115
2,1.599057e+09,CgFfWv3PUApAZUINNf,10.8.0.117,51631,8.8.8.8,53,udp,dns,0.010734,50,66,SF,0,Dd,1,78,1,94
3,1.599057e+09,CIkFu02IznJPZcp1El,10.8.0.117,65449,8.8.8.8,53,udp,dns,0.010405,37,91,SF,0,Dd,1,65,1,119
4,1.599057e+09,Cylq6E2mc9lVLjs8ua,10.8.0.117,63247,8.8.8.8,53,udp,dns,0.009332,40,80,SF,0,Dd,1,68,1,108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,1.599057e+09,CsMdO13bzHM8aWrvA9,10.8.0.117,36738,20.44.78.251,443,tcp,ssl,6.154550,177,5043,SF,0,ShADadFfR,6,509,4,1212
117,1.599057e+09,CXVBUUhqkJEhiN2s6,10.8.0.117,56248,104.244.42.194,443,tcp,ssl,359.765525,14613,6278,SF,0,ShADadTFfR,71,20987,49,8834
118,1.599057e+09,ClMierphk7pDs2oC,10.8.0.117,52184,40.74.219.49,443,tcp,ssl,96.595882,236,6037,RSTR,0,ShADFrT,14,2704,2,92
127,1.599057e+09,CmKQPy2E2AqKmpfOrd,10.8.0.117,54534,172.217.23.202,443,tcp,ssl,427.513973,9162,3885,RSTR,0,ShADadTfr,24,17163,22,5097


,ts,id.orig_p,id.resp_p,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes
count,1.020000e+02,102.000000,102.000000,102.0,102.000000,102.000000,102.000000,1.020000e+02
mean,1.599057e+09,44415.392157,277.147059,0.0,146.705882,10623.852941,277.294118,3.668732e+05
std,6.377959e+01,15755.659919,253.810462,0.0,736.495442,40890.301665,1448.574879,1.986702e+06
min,1.599057e+09,3.000000,4.000000,0.0,1.000000,59.000000,0.000000,0.000000e+00
25%,1.599057e+09,38672.500000,53.000000,0.0,1.000000,68.000000,1.000000,1.115000e+02
50%,1.599057e+09,48365.500000,443.000000,0.0,6.000000,903.000000,4.000000,8.220000e+02
75%,1.599057e+09,54231.500000,443.000000,0.0,20.750000,2771.000000,19.000000,7.447500e+03
max,1.599057e+09,65449.000000,1900.000000,0.0,6927.000000,374707.000000,13717.000000,1.879518e+07


ts               223
uid              223
id.orig_h          3
id.orig_p        170
id.resp_h         41
id.resp_p          8
proto              3
service            4
duration         189
orig_bytes       116
resp_bytes       131
conn_state         8
missed_bytes       1
history           46
orig_pkts         50
orig_ip_bytes    125
resp_pkts         46
resp_ip_bytes    133
dtype: int64

We can also generate a ydata_profiling report to understand the train data a bit more.

In [9]:
from ydata_profiling import ProfileReport
profile = ProfileReport(train_data.copy())
profile.to_file("train_data_profile.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 18/18 [00:00<00:00, 51498.96it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Now we convert respective columns to categorical, keeping in mind that we only know the categories present in the training data. We treat `id.resp_p` (responder port) as categorical rather than numeric because port numbers are identifiers, their value has no meaningful ordering for our purposes. Ref: https://docs.zeek.org/en/current/scripts/base/protocols/conn/main.zeek.html

In [10]:
df["proto"] = df["proto"].astype("category")
df["service"] = df["service"].astype("category")
df["conn_state"] = df["conn_state"].astype("category")
df["id.resp_p"] = df["id.resp_p"].astype("category")

In [11]:
train_data, test_data = train_test_split(df)

We will also include numerical data: `duration`, `orig_bytes`, `resp_bytes`, `orig_pkts`, `orig_ip_bytes`, `resp_pkts` and `resp_ip_bytes`, so we need to convert them to numeric types. We will also manufacture two additional features: `orig_bytes_per_pkt` and `resp_bytes_per_pkt`, which are the average bytes per packet for the originator and responder, respectively. We will fill missing values with 0, which is a common approach for network traffic data, as missing values often indicate no activity in that feature.

In [12]:
# convert cols to numeric, fill missing with 0
num_cols = ["duration", "orig_bytes", "resp_bytes", "orig_pkts", "orig_ip_bytes", "resp_pkts", "resp_ip_bytes"]
for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0)

df["orig_bytes_per_pkt"] = df["orig_ip_bytes"] / df["orig_pkts"].replace(0, 1)
df["resp_bytes_per_pkt"] = df["resp_ip_bytes"] / df["resp_pkts"].replace(0, 1)

num_features = ["duration", "orig_bytes", "resp_bytes", "orig_pkts", "orig_ip_bytes", "resp_pkts", "resp_ip_bytes", "orig_bytes_per_pkt", "resp_bytes_per_pkt"]
cat_features = ["proto", "service", "conn_state", "id.resp_p"]

train_data, test_data = train_test_split(df)


preprocessor = ColumnTransformer(
    transformers=[
        ("num", RobustScaler(), num_features),
        ("cat", OneHotEncoder(handle_unknown="infrequent_if_exist", min_frequency=0.01), cat_features)
    ])

X_train_raw = train_data[num_features + cat_features]
X_test_raw = test_data[num_features + cat_features]

preprocessor.fit(X_train_raw)

_X_train = preprocessor.transform(X_train_raw)
_X_test = preprocessor.transform(X_test_raw)

# make type checker happy
assert isinstance(_X_train, np.ndarray)
assert isinstance(_X_test, np.ndarray)
X_train = _X_train
X_test = _X_test

print(X_train.shape)

(102, 26)


Note that we do not process `history` here; TCP state machine transitions are something that might help us uncover TCP connections behaving unexpectedy, for example those used in volumetric attacks, but for the sake of simplicity, I decided to not include them in my model.

We train a OneClassSVM model, which learns a decision boundary around the training data (first 4 minutes known to be OK). Points predicted to the other side of this boundary will be flagged as anomalies.

As the `nu` parameter specifies the "rate of anomalies (outliers) in the training data", and we are working with data guaranteed to have no such anomalies, we want to keep the value of `nu` very low.

We perform a grid search over `nu`, `gamma`, and `kernel` type. We will use a 5Fold crossvalidation: for each fold, we take a small part of the training data out, train the model and on the part that we have taken out, we measure **recall (rate of unseen benign samples being correctly classified as not anomalous)**, which lets us to assess generalization without involving test data. Measuring recall on the whole training data and then measuring recall on the same data could lead to overfitting, and we want to avoid using test data in any way during training, as if we had not had these data yet.

Using only recall to describe the entire performance of an anomaly detector is not enough. It tells us that the model generalises well to unseen nonanomlaous data, but it cannot penalise a model, that marks too much data as anomalous (a model with the best recall could detect 0 anomalies in the test set VS one that detects everything as anomalous will be identical with this metric). But I decided to accept this limitation because without any anomaly labels, there is no way to measure the false positive rate on the test set. This metric at least ensures that our model is not entirely degenerate.

Both models use their default decision boundaries (score == 0) to separate inliers from outliers. In a real scenario this boundary could be manually shifted to adjust the "sensitivity" according to a desired tradeoff between false negatives (leading to attacks being overlooked), and false positives (leading to being overwhelmed by false alerts). The cost of missing a real attack is usually higher than the cost of investigating a false alarm. Here we leave the default because we have no labelled anomalies to calibrate against, and comparing two models at their natural boundaries already gives us sufficient information about the data.

In [13]:
from sklearn.svm import OneClassSVM

param_grid = ParameterGrid([
    {
        "nu": [0.0001, 0.001, 0.01], # train data is clean so we want to set nu to a very low value
        "gamma": ["scale", "auto", 0.0001, 0.001, 0.01, 0.1],
        "kernel": ["rbf", "poly", "linear"]
    }
])


best_svm_model = OneClassSVM()
best_svm_recall = 0.0
best_svm_params = {}

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for params in param_grid:
    fold_recalls = []
    for train_idx, val_idx in kf.split(X_train):
        X_fold_train = X_train[train_idx]
        X_fold_val = X_train[val_idx]

        model = OneClassSVM(**params)
        model.fit(X_fold_train)

        # how many validation samples correctly predicted as non-anomalous
        val_pred = model.predict(X_fold_val)
        recall = (val_pred == 1).mean()
        fold_recalls.append(recall)

    mean_recall = np.mean(fold_recalls)
    if mean_recall > best_svm_recall:
        print(f"new best CV recall {mean_recall:.4f} with SVM params {params}")
        best_svm_recall = mean_recall
        best_svm_params = params

print(f"\nSVM best params: {best_svm_params}")
print(f"SVM best CV benign recall: {best_svm_recall:.4f}")

# retrain on full training data with best params
best_svm_model = OneClassSVM(**best_svm_params)
best_svm_model.fit(X_train)

train_pred = best_svm_model.predict(X_train)
print(f"SVM train inlier rate: {(train_pred == 1).mean():.4f}")

svm_test_pred = best_svm_model.predict(X_test)
svm_anomalies = test_data[svm_test_pred == -1]

print(f"{len(svm_anomalies)}/{len(test_data)} rows from test set detected as anomalies")
display(svm_anomalies)

new best CV recall 0.3524 with SVM params {'gamma': 'scale', 'kernel': 'rbf', 'nu': 0.0001}
new best CV recall 0.5724 with SVM params {'gamma': 'scale', 'kernel': 'rbf', 'nu': 0.001}
new best CV recall 0.6243 with SVM params {'gamma': 'scale', 'kernel': 'rbf', 'nu': 0.01}
new best CV recall 0.9519 with SVM params {'gamma': 'scale', 'kernel': 'linear', 'nu': 0.0001}

SVM best params: {'gamma': 'scale', 'kernel': 'linear', 'nu': 0.0001}
SVM best CV benign recall: 0.9519
SVM train inlier rate: 0.9706
54/121 rows from test set detected as anomalies


,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,orig_bytes_per_pkt,resp_bytes_per_pkt
88,1.599057e+09,CF9VVu1vAvWE78HkF1,10.8.0.117,60754,69.171.250.20,443,tcp,ssl,0.195208,2092.0,965.0,RSTO,0,ShADadR,20,3140,16,1805,157.000000,112.812500
94,1.599058e+09,C60KUBB1mWVgTHSli,10.8.0.117,47782,147.32.83.230,8000,tcp,unknown,3.260685,0.0,0.0,S0,0,S,3,180,0,0,60.000000,0.000000
107,1.599058e+09,C7bvit4YLpboA02B71,10.8.0.117,34060,157.240.30.63,443,tcp,ssl,65.744814,123045.0,2222.0,SF,0,ShADadfFr,122,129397,81,6418,1060.631148,79.234568
120,1.599057e+09,CFtHkp1OboAxZmreoi,10.8.0.117,60756,69.171.250.20,443,tcp,ssl,74.826385,2447.0,1311.0,S1,0,ShADadt,24,3715,22,2554,154.791667,116.090909
124,1.599058e+09,C8QVv64L7m9Pcpq3d7,10.8.0.117,47782,147.32.83.230,8000,tcp,unknown,0.000000,0.0,0.0,SH,0,F,1,52,0,0,52.000000,0.000000
125,1.599058e+09,Cy8fPZ2zCP7rasvhg5,10.8.0.117,47782,147.32.83.230,8000,tcp,unknown,0.000000,0.0,0.0,SH,0,F,1,52,0,0,52.000000,0.000000
128,1.599058e+09,C0PFhIFGlyKYx7o43,10.8.0.117,47782,147.32.83.230,8000,tcp,unknown,0.000000,0.0,0.0,SH,0,F,1,52,0,0,52.000000,0.000000
129,1.599058e+09,C5xRuErcvFsJr30d1,10.8.0.117,47782,147.32.83.230,8000,tcp,unknown,0.000000,0.0,0.0,SH,0,F,1,52,0,0,52.000000,0.000000
137,1.599058e+09,CBpKqg4mytfufZvLLi,10.8.0.117,47800,147.32.83.230,8000,tcp,unknown,0.000000,0.0,0.0,SH,0,F,1,52,0,0,52.000000,0.000000
138,1.599058e+09,CdAch83rCMKFcC3Fli,10.8.0.117,47800,147.32.83.230,8000,tcp,unknown,0.000000,0.0,0.0,SH,0,F,1,52,0,0,52.000000,0.000000


We have reached a CV benign recall close to 1, meaning the model correctly classifies the vast majority of benign samples, that we held out, as inliers across all CV folds.

For comparison, we will also train an Isolation Forest. It doesn't require scaling/preprocessing features as much, but for simplicity we will train on the same data.

Choosing `contamination` to `0.05` may be a bit unexpected, as we know that the train data is entirely clean. Here it is used just to adjust the decision threshold a bit to minimize false positives on the test set.

In [14]:
from sklearn.ensemble import IsolationForest

iso_forest = IsolationForest(contamination=0.05, random_state=42, n_estimators=150)
iso_forest.fit(X_train)

iso_train_pred = iso_forest.predict(X_train)
iso_test_pred = iso_forest.predict(X_test)

print(f"IsolationForest train acc: {(iso_train_pred == 1).mean():.4f}")

iso_anomalies = test_data[iso_test_pred == -1]
print(f"{len(iso_anomalies)}/{len(test_data)} rows from test set detected as anomalies")
display(iso_anomalies)

IsolationForest train acc: 0.9412
51/121 rows from test set detected as anomalies


,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,orig_bytes_per_pkt,resp_bytes_per_pkt
94,1.599058e+09,C60KUBB1mWVgTHSli,10.8.0.117,47782,147.32.83.230,8000,tcp,unknown,3.260685,0.0,0.0,S0,0,S,3,180,0,0,60.000000,0.000000
108,1.599058e+09,CHTNtx4Fle9gvLfoDe,10.8.0.1,3,10.8.0.117,4,icmp,unknown,30.622713,3836.0,0.0,OTH,0,None,7,4032,0,0,576.000000,0.000000
124,1.599058e+09,C8QVv64L7m9Pcpq3d7,10.8.0.117,47782,147.32.83.230,8000,tcp,unknown,0.000000,0.0,0.0,SH,0,F,1,52,0,0,52.000000,0.000000
125,1.599058e+09,Cy8fPZ2zCP7rasvhg5,10.8.0.117,47782,147.32.83.230,8000,tcp,unknown,0.000000,0.0,0.0,SH,0,F,1,52,0,0,52.000000,0.000000
128,1.599058e+09,C0PFhIFGlyKYx7o43,10.8.0.117,47782,147.32.83.230,8000,tcp,unknown,0.000000,0.0,0.0,SH,0,F,1,52,0,0,52.000000,0.000000
129,1.599058e+09,C5xRuErcvFsJr30d1,10.8.0.117,47782,147.32.83.230,8000,tcp,unknown,0.000000,0.0,0.0,SH,0,F,1,52,0,0,52.000000,0.000000
131,1.599058e+09,CLfWhl4LNwbNxuP75c,10.8.0.1,3,10.8.0.117,4,icmp,unknown,7.854616,1644.0,0.0,OTH,0,None,3,1728,0,0,576.000000,0.000000
137,1.599058e+09,CBpKqg4mytfufZvLLi,10.8.0.117,47800,147.32.83.230,8000,tcp,unknown,0.000000,0.0,0.0,SH,0,F,1,52,0,0,52.000000,0.000000
138,1.599058e+09,CdAch83rCMKFcC3Fli,10.8.0.117,47800,147.32.83.230,8000,tcp,unknown,0.000000,0.0,0.0,SH,0,F,1,52,0,0,52.000000,0.000000
139,1.599058e+09,CSAhOHKnHdnB2Q73,10.8.0.117,47800,147.32.83.230,8000,tcp,unknown,0.000000,0.0,0.0,SH,0,F,1,52,0,0,52.000000,0.000000


Let us now compare the results of both models on the test set.

In [15]:
svm_anomalies_idx = set(svm_anomalies.index)
iso_anomalies_idx = set(iso_anomalies.index)

common_anomalies_idx = svm_anomalies_idx.intersection(iso_anomalies_idx)
print(f"SVM anomalies: {len(svm_anomalies_idx)}")
print(f"IsolationForest anomalies: {len(iso_anomalies_idx)}")
print(f"common anomalies: {len(common_anomalies_idx)}")

common_anomalies = test_data.loc[list(common_anomalies_idx)]

SVM anomalies: 54
IsolationForest anomalies: 51
common anomalies: 40


We can see that while both models detected slightly different data as anomalous, the vast majority of anomalies is detected by both models.

In [16]:
eval_data = test_data.copy()
eval_data["anomaly_svm"] = svm_test_pred
eval_data["anomaly_iso"] = iso_test_pred
eval_data["result_differs"] = ((eval_data["anomaly_svm"] != eval_data["anomaly_iso"]))
eval_data["is_anomaly"] = ((eval_data["anomaly_svm"] == -1) & (eval_data["anomaly_iso"] == -1))

display(eval_data[eval_data["result_differs"]])

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,...,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,orig_bytes_per_pkt,resp_bytes_per_pkt,anomaly_svm,anomaly_iso,result_differs,is_anomaly
88,1.599057e+09,CF9VVu1vAvWE78HkF1,10.8.0.117,60754,69.171.250.20,443,tcp,ssl,0.195208,2092.0,...,20,3140,16,1805,157.000000,112.812500,-1,1,True,False
107,1.599058e+09,C7bvit4YLpboA02B71,10.8.0.117,34060,157.240.30.63,443,tcp,ssl,65.744814,123045.0,...,122,129397,81,6418,1060.631148,79.234568,-1,1,True,False
108,1.599058e+09,CHTNtx4Fle9gvLfoDe,10.8.0.1,3,10.8.0.117,4,icmp,unknown,30.622713,3836.0,...,7,4032,0,0,576.000000,0.000000,1,-1,True,False
120,1.599057e+09,CFtHkp1OboAxZmreoi,10.8.0.117,60756,69.171.250.20,443,tcp,ssl,74.826385,2447.0,...,24,3715,22,2554,154.791667,116.090909,-1,1,True,False
131,1.599058e+09,CLfWhl4LNwbNxuP75c,10.8.0.1,3,10.8.0.117,4,icmp,unknown,7.854616,1644.0,...,3,1728,0,0,576.000000,0.000000,1,-1,True,False
141,1.599058e+09,CLcQhW1SxpdlwFMcna,74.125.71.188,5228,10.8.0.117,57736,tcp,unknown,0.021791,24.0,...,2,128,2,132,64.000000,66.000000,-1,1,True,False
142,1.599058e+09,C4NXeo3k9I9F6W2swk,10.8.0.117,44670,157.240.30.34,443,tcp,unknown,0.050721,31.0,...,2,135,2,135,67.500000,67.500000,-1,1,True,False
143,1.599058e+09,CMiUg3fa4nsckqYh,10.8.0.117,60690,69.171.250.20,443,tcp,unknown,0.050933,31.0,...,2,135,2,135,67.500000,67.500000,-1,1,True,False
144,1.599059e+09,ClWzPf3oMl4uVjOMT3,10.8.0.1,3,10.8.0.117,4,icmp,unknown,0.000043,1096.0,...,2,1152,0,0,576.000000,0.000000,1,-1,True,False
150,1.599059e+09,Czch711A3bfMwIhkpd,10.8.0.117,57736,74.125.71.188,5228,tcp,unknown,0.063941,28.0,...,2,132,2,130,66.000000,65.000000,-1,1,True,False


And here we show the rows where both models agree on an anomaly.

In [17]:
display(eval_data[eval_data["is_anomaly"]])

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,...,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,orig_bytes_per_pkt,resp_bytes_per_pkt,anomaly_svm,anomaly_iso,result_differs,is_anomaly
94,1.599058e+09,C60KUBB1mWVgTHSli,10.8.0.117,47782,147.32.83.230,8000,tcp,unknown,3.260685,0.0,...,3,180,0,0,60.000000,0.000000,-1,-1,False,True
124,1.599058e+09,C8QVv64L7m9Pcpq3d7,10.8.0.117,47782,147.32.83.230,8000,tcp,unknown,0.000000,0.0,...,1,52,0,0,52.000000,0.000000,-1,-1,False,True
125,1.599058e+09,Cy8fPZ2zCP7rasvhg5,10.8.0.117,47782,147.32.83.230,8000,tcp,unknown,0.000000,0.0,...,1,52,0,0,52.000000,0.000000,-1,-1,False,True
128,1.599058e+09,C0PFhIFGlyKYx7o43,10.8.0.117,47782,147.32.83.230,8000,tcp,unknown,0.000000,0.0,...,1,52,0,0,52.000000,0.000000,-1,-1,False,True
129,1.599058e+09,C5xRuErcvFsJr30d1,10.8.0.117,47782,147.32.83.230,8000,tcp,unknown,0.000000,0.0,...,1,52,0,0,52.000000,0.000000,-1,-1,False,True
137,1.599058e+09,CBpKqg4mytfufZvLLi,10.8.0.117,47800,147.32.83.230,8000,tcp,unknown,0.000000,0.0,...,1,52,0,0,52.000000,0.000000,-1,-1,False,True
138,1.599058e+09,CdAch83rCMKFcC3Fli,10.8.0.117,47800,147.32.83.230,8000,tcp,unknown,0.000000,0.0,...,1,52,0,0,52.000000,0.000000,-1,-1,False,True
139,1.599058e+09,CSAhOHKnHdnB2Q73,10.8.0.117,47800,147.32.83.230,8000,tcp,unknown,0.000000,0.0,...,1,52,0,0,52.000000,0.000000,-1,-1,False,True
140,1.599058e+09,CCB1f51bXHx5BcAKBe,10.8.0.117,47800,147.32.83.230,8000,tcp,unknown,0.000000,0.0,...,1,52,0,0,52.000000,0.000000,-1,-1,False,True
145,1.599058e+09,CNPVre20YbYQSMf7Ke,10.8.0.117,47802,147.32.83.230,8000,tcp,http,869.676223,692835.0,...,639,731444,375,20378,1144.669797,54.341333,-1,-1,False,True


To visualise the results I use t-SNE (t-distributed Stochastic Neighbor Embedding), a non-linear dimensionality reduction technique that preserves local neighborhood structure when projecting high-dimensional data into 2D. Initially I tried PCA, but it did not give me a good visual separation of normal vs anomalous points when reduced to 2 dimensions, while t-SNE does. t-SNE preserves local structure better than PCA, making it easier to see clusters and outliers if we want to make a visualisation illustrating decision boundaries among non linear data. I admit that my understanding of its inner workings is shallow, but it provides us with a useful visualisation.

In [19]:
from sklearn.manifold import TSNE

X_all = np.vstack([X_train, X_test])

tsne = TSNE(n_components=2, perplexity=30, random_state=42, n_iter_without_progress=1000)
X_all_tsne = tsne.fit_transform(X_all)

X_train_tsne = X_all_tsne[:X_train.shape[0]]
X_test_tsne = X_all_tsne[X_train.shape[0]:]

def get_label(svm_pred, iso_pred):
    if svm_pred == -1 and iso_pred == -1:
        return "both anomaly"
    elif svm_pred == 1 and iso_pred == 1:
        return "no anomaly"
    else:
        return "one model anomaly"

# test data
df_test_tsne = pd.DataFrame(X_test_tsne, columns=["dim1", "dim2"])
df_test_tsne["anomaly_label"] = [
    get_label(s, i) for s, i in zip(svm_test_pred, iso_test_pred)
]
df_test_tsne["uid"] = test_data["uid"].values
df_test_tsne["id.resp_p"] = test_data["id.resp_p"].values

# train data
df_train_tsne = pd.DataFrame(X_train_tsne, columns=["dim1", "dim2"])
df_train_tsne["anomaly_label"] = "train"
df_train_tsne["uid"] = train_data["uid"].values
df_train_tsne["id.resp_p"] = train_data["id.resp_p"].values

df_tsne = pd.concat([df_train_tsne, df_test_tsne], ignore_index=True)

fig = px.scatter(
    df_tsne, 
    x="dim1", 
    y="dim2", 
    color="anomaly_label",
    color_discrete_map={
        "train": "blue",
        "no anomaly": "green",
        "one model anomaly": "orange",
        "both anomaly": "red"
    },
    title="2D t-SNE visualization of train & test data with anomaly labels",
    hover_data={"uid": True, "id.resp_p": True}
)

fig.show()

Below is the same t-SNE visualisation in 3D. The extra dimension can sometimes reveal cluster structure that gets collapsed in 2D projections, and the interactive plot allows rotating the view to inspect the data from different angles.

In [20]:
tsne_3d = TSNE(n_components=3, perplexity=30, random_state=42, n_iter_without_progress=1000)
X_all_tsne_3d = tsne_3d.fit_transform(X_all)

X_train_tsne_3d = X_all_tsne_3d[:X_train.shape[0]]
X_test_tsne_3d = X_all_tsne_3d[X_train.shape[0]:]

# test data
df_test_3d = pd.DataFrame(X_test_tsne_3d, columns=["dim1", "dim2", "dim3"])
df_test_3d["anomaly_label"] = [
    get_label(s, i) for s, i in zip(svm_test_pred, iso_test_pred)
]
df_test_3d["uid"] = test_data["uid"].values
df_test_3d["id.resp_p"] = test_data["id.resp_p"].values

# train data
df_train_3d = pd.DataFrame(X_train_tsne_3d, columns=["dim1", "dim2", "dim3"])
df_train_3d["anomaly_label"] = "train"
df_train_3d["uid"] = train_data["uid"].values
df_train_3d["id.resp_p"] = train_data["id.resp_p"].values

df_tsne_3d = pd.concat([df_train_3d, df_test_3d], ignore_index=True)

fig_3d = px.scatter_3d(
    df_tsne_3d,
    x="dim1",
    y="dim2",
    z="dim3",
    color="anomaly_label",
    color_discrete_map={
        "train": "blue",
        "no anomaly": "green",
        "one model anomaly": "orange",
        "both anomaly": "red"
    },
    title="3D t-SNE visualization of train & test data with anomaly labels",
    hover_data={"uid": True, "id.resp_p": True},
    opacity=0.3
)

fig_3d.update_traces(marker=dict(size=10))
fig_3d.update_layout(
    height=800,
)

fig_3d.show()

We now inspect the anomaly score distributions produced by each model. The `decision_function` returns a distance from the decision boundary. Positive values are inliers (non-anomalous), negative values are outliers (anomalies), and zero is our threshold. The shape of the distribution shows how confidently each model separates normal from anomalous traffic.

In [21]:
svm_scores = best_svm_model.decision_function(X_test)
iso_scores = iso_forest.decision_function(X_test)

def plot_hist_distribution(scores, title):
    fig = px.histogram(
        x=scores, 
        nbins=100,
        title=title,
        labels={"x": "anomaly score"},
    )
    fig.add_vline(x=0, annotation_text="0 = threshold")
    fig.update_layout(showlegend=False)
    fig.show()

plot_hist_distribution(svm_scores, "OneClassSVM anomaly score distribution")
plot_hist_distribution(iso_scores, "IsolationForest anomaly score distribution")

We do have two significant outliers in the SVM scores, let's see what they are, and then try to visualise the distribution without it to get a better sense of the overall distribution.

In [22]:
outl1, outl2 = np.argsort(svm_scores)[:2]
display(test_data.iloc[[outl1, outl2]])

plot_hist_distribution(svm_scores[svm_scores > svm_scores[outl2]], "OneClassSVM anomaly score distribution without significant outliers")

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,orig_bytes_per_pkt,resp_bytes_per_pkt
145,1.599058e+09,CNPVre20YbYQSMf7Ke,10.8.0.117,47802,147.32.83.230,8000,tcp,http,869.676223,692835.0,810.0,RSTR,0,ShADdTawwfr,639,731444,375,20378,1144.669797,54.341333
107,1.599058e+09,C7bvit4YLpboA02B71,10.8.0.117,34060,157.240.30.63,443,tcp,ssl,65.744814,123045.0,2222.0,SF,0,ShADadfFr,122,129397,81,6418,1060.631148,79.234568


We can notice that those oultiers have unusually large sizes (`orig_bytes`, `orig_ip_bytes` etc.), which may be an indicator of data exfiltration. If that is the case adjusting our classification threshold we could modify our predictors to only detect such values. But without any anomaly labels, we cannot be sure.

At the end we will try to assess feature importance. We fit PCA on the preprocessed training data. t-SNE is non-linear, which means it does not provide us interpretability of the component loadings, which means there is no way to map its output dimensions back to original features. Compared to that, PCA components are linear combinations of the original features, which means we can directly discover which features contribute the most variance.

In [23]:
from sklearn.decomposition import PCA

feature_names = preprocessor.get_feature_names_out()

pca = PCA(n_components=2)
pca.fit(X_train)

pca_components_df = pd.DataFrame(
    data=pca.components_, 
    columns=feature_names, 
    index=["PC1", "PC2"]
)

print(f"explained variance ratio: PC1={pca.explained_variance_ratio_[0]:.4f}, PC2={pca.explained_variance_ratio_[1]:.4f}")

print("top features contributing to PC1:")
print(pca_components_df.iloc[0].abs().nlargest(10))

print("top features contributing to PC2:")
print(pca_components_df.iloc[1].abs().nlargest(10))

explained variance ratio: PC1=0.9997, PC2=0.0002
top features contributing to PC1:
num__resp_bytes            0.733250
num__resp_ip_bytes         0.645332
num__resp_pkts             0.191754
num__orig_pkts             0.088640
num__orig_ip_bytes         0.035430
num__orig_bytes            0.002532
num__resp_bytes_per_pkt    0.001746
num__orig_bytes_per_pkt    0.000439
cat__service_ssl           0.000211
cat__id.resp_p_443         0.000208
Name: PC1, dtype: float64
top features contributing to PC2:
num__orig_bytes            0.603891
num__orig_ip_bytes         0.530704
num__orig_pkts             0.345753
num__resp_bytes            0.299715
num__resp_ip_bytes         0.211056
num__duration              0.182151
num__resp_pkts             0.169006
num__orig_bytes_per_pkt    0.135646
num__resp_bytes_per_pkt    0.102147
cat__id.resp_p_443         0.041970
Name: PC2, dtype: float64


PC1 alone captures 99.97% of variance, and is dominated by `resp_bytes` and `resp_ip_bytes`. This means the training data's variance is almost entirely driven by response traffic volume. This aligns with the histogram outliers we found earlier. It also suggests that most other features (categorical columns, duration, originator bytes) contribute very little variance after scaling, which is expected given that the majority of benign connections have similar structure and differ mainly in how much data the server sends back.

## Summary

We trained two unsupervised anomaly detection models (`OneClassSVM`, `IsolationForest`) on the first 4 minutes of network traffic, and evaluated them on the remaining traffic.

##### Preprocessing
We selected numerical features (duration, bytes, packets) and categorical features (protocol, service, connection state, responder port). We engineered two additional features (bytes per packet) and used `RobustScaler` for numeric features and `OneHotEncoder` for categoricals, fitting only on training data to avoid leaking any information from the test set.

##### Model selection
We used 5 fold crossvalidation with "benign recall" as a metric to tune OneClassSVM hyperparameters via grid search (small `nu`, `gamma`, `kernel`). Then we also trained an `IsolationForest`, with `contamination=0.05` as a threshold adjustment.

##### Evaluation
We compared the two models by inspecting their agreement on detected anomalies. Flows marked as anomalous by both predictors are candidates with highest confidence. We visualised the data using t-SNE and inspected anomaly score distributions. We also used PCA for feature importance analysis.

##### Threshold
Both models use their default decision boundary (score = 0). We did not tune this threshold because we have no labelled anomalies to calibrate against. In a real deployment, the threshold could be shifted toward fewer false negatives (more alerts, fewer missed attacks) depending on real data and tolerance for false alarms.

##### Limitations
Without ground truth labels for the test data, we cannot compute precision or recall on the actual anomalies. Our CV metric ensures the model generalises to unseen non-malicious traffic, but it cannot penalise false positives (under our metric, a model that flags many test flows as anomalous scores the same as one that flags few if they just correctly classify the data being held out). The dataset is very small, having more data would definitely help. There are also many more opportunities for preprocessing and feature engineering (for example, the `history` field was not used but could provide useful information), as well as utilizing other Zeek files than `conn.log`.